In [ ]:
import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import Utils.DiaSourceTools as DSTools

In [144]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_sim/"

In [145]:
ccd = 14
parameters_order = range(0,9)
directory = DATADIR+"/deepDiffSims/"
butler = dafPersist.Butler(directory) 
visit = 850586

diaSrc = butler.get("deepDiff_diaSrc", {"ccd":ccd,"filter":'r', "visit":visit})
calexp = butler. get("calexp", {"ccd":ccd,"filter":'r', "visit":visit})
class_diaSrc = butler.get("deepDiff_class_diaSrc", {"ccd":ccd,"filter":'r', "visit":visit})
diffexp = butler.get("deepDiff_differenceExp", {"ccd":ccd,"filter":'r', "visit":visit})

In [146]:
for source in diaSrc:
    coord_ra, coord_dec = source["coord_ra"], source["coord_dec"]

In [147]:
len(diaSrc), len(class_diaSrc)

(5380, 1041)

In [140]:
import pickle

out = open("/renoir_data_02/jpreyes/lsst_data/calexp_coords.pickle", "rb")
coords = pickle.load(out)

In [141]:
wcs = calexp.getWcs()
detected_1 = []
count = 0
for source in diaSrc:
    for x, y in coords:
        radec = wcs.pixelToSky(x,y)
        s1 = {"ra": source["coord_ra"], "dec": source["coord_dec"]}
        s2 = {"ra": radec.getLongitude(), "dec": radec.getLatitude()}
        if DSTools.source_distance(s1, s2) < 1.0:
            count+=1
            detected_1.append( wcs.skyToPixel(source["coord_ra"], source["coord_dec"]) )
            break
            
            
print count
    



764


In [148]:
count = 0
detected_2=[]
for source in class_diaSrc:
    for x, y in coords:
        radec = wcs.pixelToSky(x,y)
        s1 = {"ra": source["coord_ra"], "dec": source["coord_dec"]}
        s2 = {"ra": radec.getLongitude(), "dec": radec.getLatitude()}
        if DSTools.source_distance(s1, s2) < 1.0:
            count+=1
            detected_2.append( wcs.skyToPixel(source["coord_ra"], source["coord_dec"]) )
            break
            
print count
    

978


In [106]:
import lsst.afw.display.ds9 as ds9
ds9.mtv(calexp.getMaskedImage().getImage())

In [107]:
import lsst.afw.display.ds9 as ds9
f=2
ds9.mtv(diffexp, frame=f)

In [108]:

for x, y in coords:
    ds9.dot("o", x, y, ctype=ds9.GREEN, frame=f)


In [109]:
for x,y in detected_1:
    ds9.dot("x", x, y, ctype=ds9.RED, frame=f)

In [110]:
for x,y in detected_2:
    ds9.dot("0", x, y, ctype=ds9.BLUE, frame=f)

In [111]:
for source in diaSrc:
     x,y = wcs.skyToPixel(source["coord_ra"],source["coord_dec"])
     ds9.dot("x", x, y, ctype=ds9.RED, frame=f)  

In [112]:
for source in class_diaSrc:
     x,y = wcs.skyToPixel(source["coord_ra"],source["coord_dec"])
     ds9.dot("1", x, y, ctype=ds9.MAGENTA, frame=f)

In [78]:
len(class_diaSrc)

1066